# **EV quantification in droplet**
Authors: Minato Yamashita, Kazuki Hattori\
4/7/22\
Ver.3.4

## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage.io import imread
from skimage.filters import median, gaussian, threshold_otsu
from skimage.segmentation import clear_border, watershed, relabel_sequential
from skimage.feature import peak_local_max
from skimage.measure import regionprops_table
from skimage.color import label2rgb
import random
import datetime
import pytz

## Basic information

In [ ]:
exp_id = "XXXX"
main_path = "XXXX"
file_name_front = "XXXX"
image_id_list = [format(n,"02") for n in np.arange(0,32,1)]
time_list = [format(i,"02") for i in np.arange(0,19,1)]
time_interval = 2
total_time_point = 19

## Functions

In [ ]:
#for calculating cell area
def cell_area(droplet_props, droplet_labels, cell_threshed):
    cell_areas = [np.count_nonzero(cell_threshed[droplet_labels==i]) for i in droplet_props['label']]
    df = droplet_props.copy()
    df['cell_area'] = cell_areas
    
    return df

#for selecting single cell droplets baased on cell area
def single_cell(df, thresh=(None, None)):
    df = df.copy()
    df = df[(thresh[0]<df['cell_area']) & (df['cell_area']<thresh[1])]

    return df

#for calculating average intensity of Sytox Orange
def sytox(df, droplet_label, cell_thresh, sytox):
    sytox_means = [np.mean(sytox[(droplet_label==i) & cell_thresh]) for i in df['label']]
    df_sytox = df.copy()
    df_sytox['sytox'] = sytox_means
    
    return df_sytox

#for selecting droplets with live cells
def live_cell(df, thresh):
    df_live = df.copy()
    df_live = df_live[df_live['sytox']<thresh]
    
    return df_live

#for calculating average GFP intensity outside of cell region
def droplet_intensity(df, droplet_label, green, cell_thresh):
    droplet_int = [np.mean(green[(droplet_label==i) & ~cell_thresh]) for i in df['label']]
    df_droplet_int = df.copy()
    df_droplet_int['droplet_mean_intensity'] = droplet_int
    
    return df_droplet_int

## Loop (Load, Analyze, Summarize, Save figures)

In [ ]:
#reset dataframes
res_all = pd.DataFrame()
df_droplet_cell_sum = pd.DataFrame()

for h in image_id_list:
    for i in time_list: 
        #time_point
        time_point = str(int(i) * time_interval)

        #file name of phase contrast
        file_name = file_name_front + str(i) + "_0_A01f" + h + "d4.TIF"

        #assign file name
        phase = main_path + file_name

        file_name_list_g = list(file_name)
        file_name_list_g[-5] = "1"
        file_name_list_g = "".join(file_name_list_g)
        green = main_path + file_name_list_g

        file_name_list_r = list(file_name)
        file_name_list_r[-5] = "2"
        file_name_list_r = "".join(file_name_list_r)
        red = main_path + file_name_list_r 

        #load files
        img_p = imread(phase, plugin = 'pil')
        img_g = imread(green, plugin = 'pil')
        img_r = imread(red, plugin = 'pil')

        #combine images
        img = np.stack([img_p, img_g, img_r])

        ##Analyze Phase Contrast
        #denoise
        p_f = median(img[0])
        p_f = gaussian(p_f, sigma=1)

        #binarization
        p_thresh = threshold_otsu(p_f)
        p_t = p_f > p_thresh

        #fill holes
        p_t_f = ndi.binary_fill_holes(p_t)

        #clear border
        p_t_f = clear_border(p_t_f)

        #label
        p_label, _ = ndi.label(p_t_f)

        #analyze each unit
        df = pd.DataFrame(regionprops_table(p_label, img[1], properties=('label', 'area')))

        #select labels
        labels_retained = np.array(df['label'][(df['area']>7000) & (df["area"]<12000)]) #select unit based on area
        p_label_f = np.where(np.isin(p_label, labels_retained), p_label, 0) #delete others
        p_label_f, _, _ = relabel_sequential(p_label_f) #re-label
        df_f = pd.DataFrame(regionprops_table(p_label_f, img[1], properties=('label', 'area', 'centroid', 'mean_intensity'))) #re-analyze

        #add exp/image ID
        df_f.insert(loc = 0, column= 'time', value= time_point)
        df_f.insert(loc = 0, column= 'image_id', value= h)
        df_f.insert(loc = 0, column= 'exp_id', value= exp_id)

        ##Analyze cell region, similar method to phase contrast analysis
        g_f = median(img_g)
        g_f = gaussian(g_f, sigma=1)

        g_thresh = threshold_otsu(g_f)
        g_t = g_f > g_thresh

        g_t = ndi.binary_fill_holes(g_t)

        df_droplet_cell = cell_area(df_f, p_label_f, g_t)

        #calculate average intensity of Sytox Orange
        df_droplet_cell = sytox(df_droplet_cell, p_label_f, g_t, img[2])
        df_droplet_cell = df_droplet_cell.fillna(0)
        
        #calculate GFP signals outside of cell region
        df_droplet_cell = droplet_intensity(df_droplet_cell, p_label_f, img[1], g_t)

        #analysis summary
        p_color = label2rgb(p_label_f, bg_label=0)

        fig, ax = plt.subplots(2, 2, figsize=(14, 10))
        ax[0][0].imshow(img_p)
        ax[0][0].axis('off')
        ax[0][0].set_title('Phase Contrast')
        ax[0][1].imshow(p_color)
        ax[0][1].axis('off')
        ax[1][0].imshow(g_t)
        ax[1][0].axis('off')
        ax[1][0].set_title('GFP_Cell')
        ax[1][1].imshow(img_r)
        ax[1][1].axis('off')
        ax[1][1].set_title('Sytox Orange')

        for j, l in enumerate(df_f['label']):
            ax[0][0].text(x=df_f['centroid-1'][j] + 35, y=df_f['centroid-0'][j] + 35, s=l, size=3, color='white')

        for j, l in enumerate(df_f['label']):
            ax[1][0].text(x=df_f['centroid-1'][j], y=df_f['centroid-0'][j], s=l, size=7, color='white')

        for j, l in enumerate(df_f['label']):
            ax[0][1].text(x=df_f['centroid-1'][j] + 20, y=df_f['centroid-0'][j] + 20, s=l, size=5, color='white')

        for j, l in enumerate(df_f['label']):
            ax[1][1].text(x=df_f['centroid-1'][j] + 20, y=df_f['centroid-0'][j] + 20, s=l, size=5, color='white')

        plt.tight_layout()
        plt.annotate(exp_id + "_" + h + "_" + time_point + "h", (0,2.1), xycoords='axes fraction', fontsize = 20)

        #save images
        fig.savefig(("XXXX"),dpi = 300, transparent = True, bbox_inches = "tight")
        plt.close()
        
        #select droplets with single cells
        df_droplet_single_cell = single_cell(df_droplet_cell, thresh=(300, 900))

        #select droplets with single live cells
        df_droplet_single_live_cell = live_cell(df_droplet_single_cell, 100)

        #overwrite dataframes
        res = droplet_intensity(df_droplet_single_live_cell, p_label_f, img[1], g_t)
        res_all = pd.concat([res_all, res])

        df_droplet_cell_sum = pd.concat([df_droplet_cell_sum, df_droplet_cell], ignore_index = True) #concatenate all data


## Export data table

In [ ]:
res_all.to_excel("XXXX", sheet_name = "single_cell")
df_droplet_cell_sum.to_excel("AAAA", sheet_name = "all")

## Select droplets that were successfully tracked throughout the entire period (for background calculation)

In [ ]:
data_back_sum = pd.DataFrame()
data_image_sum = pd.DataFrame()
image_id_list2 = np.arange(0,32)

data = pd.read_excel("AAAA")

data_back = data[(data["cell_area"] == 0) & (data["sytox"] == 0)]
data_back = data_back.reset_index(drop = True)

#select tracked droplets without cells
for j in image_id_list2:
  data_back_image = pd.DataFrame()
  data_back_selected = data_back[data_back["image_id"] == j]
  data_back_t0 = data_back_selected[data_back_selected["time"] == 0].copy()
  data_back_t0 = data_back_t0.reset_index(drop = True)

  for i in range(len(data_back_t0)):
    x = data_back_t0.loc[i,"centroid-0"]
    y = data_back_t0.loc[i,"centroid-1"]
    data_back_tracked = data_back_selected[(data_back_selected["centroid-0"] > x - 50) & (data_back_selected["centroid-0"] < x + 50) & (data_back_selected["centroid-1"] > y - 50) & (data_back_selected["centroid-1"] < y + 50)].copy()

    if len(data_back_tracked) == total_time_point:
        pass
    else:
        data_back_tracked = pd.DataFrame()
        
    data_back_image = pd.concat([data_back_image, data_back_tracked], ignore_index = True)

  data_back_sum = pd.concat([data_back_sum, data_back_image], ignore_index = True)

data_back_sum.to_excel("XXXX")


for j in np.arange(0, time_interval * total_time_point, time_interval):
  data_image_each_time_point = pd.DataFrame()
  
  for i in image_id_list2:

    #background calculation
    data_back_image = data_back_sum[(data_back_sum["image_id"] == i) & (data_back_sum["time"] == j)]
    back_mean = data_back_image["mean_intensity"].mean()
    back_sd = data_back_image["mean_intensity"].std()
    data_back_image = data_back_image[(data_back_image["mean_intensity"] > back_mean - 2 * back_sd) & (data_back_image["mean_intensity"] < back_mean + 2 * back_sd)].copy()
    back = data_back_image["mean_intensity"].median()

    #background subtraction in each image
    data_image = data[(data["image_id"] == i) & (data["time"] == j)].copy()
    data_image['back'] = np.repeat(back, len(data_image))
    data_image['droplet_mean_intensity_minus_back'] = data_image["droplet_mean_intensity"] - back

    data_image_each_time_point = pd.concat([data_image_each_time_point, data_image], ignore_index = True)
  data_image_sum = pd.concat([data_image_sum, data_image_each_time_point], ignore_index = True)

data_image_sum.to_excel("BBBB")

## Select droplets that were successfully tracked throughout the entire period (for droplets with live cells)

In [ ]:
data_single_sum = pd.DataFrame()
image_id_list2 = np.arange(0,32)
cell_area_thres = 1200
  
#import data
data = pd.read_excel("BBBB")

#select droplets with live cells
data_cell = data[(data["cell_area"] > 300) & ((data["sytox"] < 100))]
data_cell = data_cell.reset_index(drop = True)

for j in image_id_list2:
  data_single_image = pd.DataFrame()
  data_cell_selected = data_cell[data_cell["image_id"] == j]
  data_t0 = data_cell_selected[data_cell_selected["time"] == 0].copy()
  data_t0 = data_t0.reset_index(drop = True)

  for i in range(len(data_t0)):
    x = data_t0.loc[i,"centroid-0"]
    y = data_t0.loc[i,"centroid-1"]
    data_single = data_cell_selected[(data_cell_selected["centroid-0"] > x - 50) & (data_cell_selected["centroid-0"] < x + 50) & (data_cell_selected["centroid-1"] > y - 50) & (data_cell_selected["centroid-1"] < y + 50)].copy()
 
    if(data_single[data_single["time"] == 0]["cell_area"].values < cell_area_thres):
        pass
    else:
        data_single = pd.DataFrame()

    if len(data_single) == total_time_point:
        pass
    else:
        data_single = pd.DataFrame()
        
    data_single_image = pd.concat([data_single_image, data_single], ignore_index = True)

  data_single_sum = pd.concat([data_single_sum, data_single_image], ignore_index = True)

total_cell_no = len(data_single_sum)/ total_time_point
data_single_sum["cell_id"] = np.repeat(np.arange(1,total_cell_no + 1), 19)

#save dataframe
data_single_sum.to_excel("XXXX")


# Save summary

In [ ]:
lines = ["",
         "Exp.ID: " + exp_id, 
         "image#_used: " + str(len(image_id_list)),
         "imageID_used: " + str(image_id_list),
         "droplet#_selected: " + str(len(data_single_sum)/total_time_point), 
         "cell_area_threshold_at_t0: 300> and <" + str(cell_area_thres),
         "time_interval: " + str(time_interval),
         "total_time_point: " + str(total_time_point),
         "duration [h]: " + str(time_interval * (total_time_point - 1)),
         "time_stamp: " + str(datetime.datetime.now(pytz.timezone('Asia/Tokyo')))]
with open("XXXX", 'a') as f:
    for line in lines:
        f.write(line)
        f.write('\n')